In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re

In [2]:
df = pd.read_csv('baseDeDados.csv', sep=';')

/media/user/DADOS1/anaconda/envs/iainverno/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def extend_list(lst):
    try:
        lst[2]
    except:
        lst.append('')
    return lst

def list_to_int(lst):
    new_lst = []
    for item in lst:
        try:
            new_lst.append(int(item))
        except:
            continue
    return new_lst

def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1       
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

class Knapsack(object):
    
    def __init__(self, C, weights, values, names):
        self.C = C
        self.weights = weights
        self.values = values
        self.names = names
        self.K = None
        
    def run(self):
        self.K = [ [0 for x in range(self.C+1)] for x in range(len(self.values)+1) ]
        for i in range(len(self.values)+1):
            for j in range(self.C+1):
                if i==0 or j==0:
                    self.K[i][j] = 0
                elif self.weights[i-1] <= j:
                    self.K[i][j] = max( self.values[i-1] + self.K[i-1][j - self.weights[i-1] ], self.K[i-1][j] )
                else:
                    self.K[i][j] = self.K[i-1][j]
        return self
    
    def get(self):
        items = []
        y = K[len(self.values)][self.C]
        z = C
        for i in range(len(self.values), 0, -1):
            if y <= 0:
                break
            if z == self.K[i - 1][z]:
                continue
            else:
                items.append(self.names[i - 1])
                y = y - self.values[i - 1]
                z = z - self.weights[i - 1]
        return items

def tabela_notas(df):
    return df[['nome_componente', 'nota_discente_componente']]\
    .groupby('nome_componente')\
    .mean()\
    .to_dict()\
    ['nota_discente_componente']

def tabela_dependencias_grade_nova(df):
    # Copiado de http://inf.ufg.br/sites/default/files/uploads/Resolucao_CEPEC_2017_1561.pdf
    novos_requisitos = """
    01 Álgebra Linear IME 64 0 64 NC OBR
    02 Algoritmos e Estruturas de Dados 1 INF 26 32 32 64 NC OBR
    03 Algoritmos e Estruturas de Dados 2 INF 2 64 0 64 NC OBR
    04 Análise e Projeto de Algoritmos INF 21, 03 64 0 64 NC OBR
    05 Arquitetura de Computadores INF 21 48 16 64 NC OBR
    06 Banco de Dados INF 29 48 16 64 NC OBR
    07 Cálculo 1A IME 96 0 96 NC OBR
    08 Cálculo 2A IME 7 96 0 96 NE OBR
    09 Cálculo Numérico IME 18 64 0 64 NE OBR
    10 Circuitos Digitais EMC 21 64 0 64 NE OBR
    11 Compiladores INF 03, 28 48 16 64 NE OBR
    12 Computação e Sociedade INF 32 0 32 NC OBR
    13 Computação Gráfica INF 22 32 32 64 NE OBR
    14 Computação Paralela INF 39 48 16 64 NE OBR
    15 Eletrônica para Computação EMC 20 96 0 96 NE OBR
    16 Engenharia de Requisitos INF 16 48 64 NE OBR
    17 Engenharia de Software INF 64 0 64 NC OBR
    18 Equações Diferenciais Ordinárias IME 01, 08 64 0 64 NE OBR
    19 Física I IF 7 64 0 64 NE OBR
    20 Física III IF 19 64 0 64 NE OBR
    21 Fundamentos de Matemática para Computação INF 64 0 64 NC OBR
    22 Geometria Analítica IME 64 0 64 NE OBR
    23 Inteligência Artificial INF 02, 29, 33 64 0 64 NE OBR
    24 Interação Humano-Computador INF 02, 33 32 32 64 NC OBR
    25 Introdução à Computabilidade e à Complexidade Computacional INF 04, 28 64 0 64 NE OBR
    26 Introdução à Programação INF 48 80 128 NC OBR
    27 Linguagens e Paradigmas de Programação INF 26 32 32 64 NC OBR
    28 Linguagens Formais e Autômatos INF 21 64 0 64 NE OBR
    29 Lógica Matemática INF 21 64 0 64 NC OBR
    30 Matemática Discreta INF 21 64 0 64 NE OBR
    31 Metodologia de Pesquisa em Computação INF 32 0 32 NE OBR
    32 Pesquisa Operacional INF 1 64 0 64 NE OBR
    33 Probabilidade e Estatística A IME 7 64 0 64 NC OBR
    34 Programação Orientada a Objetos INF 26 32 32 64 NC OBR
    35 Projeto de Software INF 17, 34 64 0 64 NC OBR
    36 Projeto Final de Curso 1 INF 0 64 64 NE OBR
    37 Projeto Final de Curso 2 INF 36 0 64 64 NE OBR
    38 Redes de Computadores 1 INF 48 16 64 NE OBR
    39 Sistemas Distribuídos INF 38 48 16 64 NE OBR
    40 Sistemas Gerenciadores de Banco de Dados INF 6 64 0 64 NE OBR
    41 Sistemas Operacionais INF 02, 05 64 0 64 NE OBR
    42 Software Básico INF 2 0 64 64 NE OBR
    43 Teoria dos Grafos INF 30 64 0 64 NE OBR
    44 Aprendizagem de Máquina e Mineração de Dados INF 23 32 32 64 NE OPT
    45 Armazém de Dados INF 40 48 16 64 NE OPT
    46 Avaliação de Desempenho de Sistemas Computacionais INF 33 64 0 64 NE OPT
    47 Banco de Dados Distribuídos INF 40 64 0 64 NE OPT
    48 Computação Evolucionária INF 32 32 64 NE OPT
    49 Computação Móvel e Ubíqua INF 34, 35 16 48 64 NE OPT
    50 Computação Verde INF 38 32 32 64 NE OPT
    51 Desenvolvimento de Sistemas para Web INF 34 32 32 64 NE OPT
    52 Desenvolvimento de Software Dirigido por Modelos INF 02, 34 32 32 64 NE OPT
    53 Empreendedorismo Digital INF 32 32 64 NE OPT
    54 Engenharia de Sistemas INF 64 0 64 NE OPT
    55 Engenharia de Software Baseada em Busca INF 17 32 32 64 NE OPT
    56 Fundamentos da Programação Lógica e Funcional INF 27, 29 64 0 64 NE OPT
    57 Integração de Dados INF 40 64 0 64 NE OPT
    58 Introdução à Língua Brasileira de Sinais FL 64 0 64 NE OPT
    59 Introdução à Teoria das Provas INF 29 64 0 64 NE OPT
    60 Introdução à Teoria dos Tipos INF 29 64 0 64 NE OPT
    61 Jogos Digitais INF 34 16 48 64 NE OPT
    62 Meta-heurísticas INF 2 64 0 64 NE OPT
    63 Metodologia e Experimentação em Engenharia de Software INF 64 0 64 NE OPT
    64 Microcontroladores INF 10, 26 32 32 64 NE OPT
    65 Processamento Digital de Imagens INF 01, 26 64 0 64 NE OPT
    66 Recuperação de Informação INF 02, 33 64 0 64 NE OPT
    67 Redes de Computadores 2 INF 38 32 32 64 NE OPT
    68 Redes Neuronais Artificiais INF 23 32 32 64 NE OPT
    69 Segurança e Auditoria de Sistemas INF 38 32 32 64 NE OPT
    70 Sistemas Multiagentes INF 34 32 32 64 NE OPT
    71 Tópicos Avançados em Redes Sem Fio INF 16 48 64 NE OPT
    72 Tópicos em Computação INF 64 0 64 NE OPT
    73 Tópicos em Teste de Software INF 48 16 64 NE OPT
    74 Verificação Formal de Programas INF 29 32 32 64 NE OPT
    75 Visão Robótica INF 01, 65 64 0 64 NE OPT
    76 Visualização de Informações INF 3 32 32 64 NE OPT
    77 Web Semântica INF 34 32 32 64 NE OPT
    """.split('\n')[1:-1]

    novos_requisitos = [ re.sub('[0-9]* [0-9]* [0-9]* [A-Z]* [A-Z]*$', '', line) for line in novos_requisitos ]

    novos_requisitos = [ re.split('(^[0-9]* )', line)[1:] for line in novos_requisitos ]

    novos_requisitos = [ [x[0]] + re.split(' [A-Z][A-Z][A-Z]? ', x[1]) for x in novos_requisitos ]

    novos_requisitos = [ extend_list(x) for x in novos_requisitos ]

    novos_requisitos = [ x[1:] for x in novos_requisitos ]
    
    for idx, item in enumerate(novos_requisitos):
        tmp = []
        p1 = item[0]
        p2 = item[1]
        p1 = p1.lstrip()
        p1 = p1.split(' ', 1)
        tmp.extend(p1)
        tmp.append(p2)
        if len(tmp) == 2:
            tmp.append([])
        novos_requisitos[idx] = tmp
              
    novos_requisitos = [
        {
            "key": int(x[0]),
            "name": x[1],
            "deps": list_to_int(x[2].split(','))
        } for x in novos_requisitos
    ]
    
    materias_tabela = df[df['codigo_matriz_curricular']=='CICOMP-BI-3']['nome_componente'].drop_duplicates().values

    grafo_requisitos = []
    for x in materias_tabela:
        distances = []
        names = []
        for y in novos_requisitos:
            distances.append( levenshtein(x.lower(),y['name'].lower()) )
            names.append(y['name'])
        match = names[distances.index(min(distances))]
        for y in novos_requisitos:
            if y['name'] == match:
                y['name'] = x
                grafo_requisitos.append(y)

    for item in grafo_requisitos:
        deps = item['deps']
        for idx,dep in enumerate(deps):
            for item2 in grafo_requisitos:
                if item2['key'] == dep:
                    item['deps'][idx] = item2['name']

    dependencias_grade_nova = []
    for item in grafo_requisitos:
        for dep in item['deps']:
            row = {
                "materia": item['name'],
                "dependencia": dep
            }
            dependencias_grade_nova.append(row)
    dependencias_grade_nova = pd.DataFrame(dependencias_grade_nova)
    return dependencias_grade_nova

def tabela_dependencias_grade_antiga(df):
    # Copiado de http://www.inf.ufg.br/sites/default/files/ProjetoPedagogico2008.pdf
    requisitos = [
        {
        "materia": "Programação de Computadores 2",
        "dependencia": "Programação de Computadores 1"
    },
        {
        "materia": "Estrutura de Dados I",
        "dependencia": "Programação de Computadores 1"
    },
        {
        "materia": "Programação Orientada a Objetos",
        "dependencia": "Programação de Computadores 1"
    },
        {
        "materia": "Linguagens de Programação",
        "dependencia": "Programação de Computadores 1"
    },
        {
        "materia": "Estruturas de Dados II",
        "dependencia": "Estruturas de Dados I"
    },
        {
        "materia": "Análise e Projeto de Algoritmos",
        "dependencia": "Estruturas de Dados I"
    },
        {
        "materia": "Teoria da Computação",
        "dependencia": "Linguagens Formais e Autômatos"
    },
        {
        "materia": "Compiladores",
        "dependencia": "Linguagens Formais e Autômatos"
    },    
        {
        "materia": "Compiladores",
        "dependencia": "Estruturas de Dados II"
    },
        {
        "materia": "Redes de Computadores 2",
        "dependencia": "Redes de Computadores 1"
    },
        {
        "materia": "Sistemas Distribuídos",
        "dependencia": "Redes de Computadores 1"
    },
        {
        "materia": "Inteligência Artificial",
        "dependencia": "Lógica Matemática"
    },
        {
        "materia": "Banco de Dados",
        "dependencia": "Lógica Matemática"
    },
        {
        "materia": "Sistemas Gerenciadores de BD",
        "dependencia": "Banco de Dados"
    },
        {
        "materia": "Sistemas Operacionais 2",
        "dependencia": "Sistemas Operacionais 1"
    },
        {
        "materia": "Cálculo 2",
        "dependencia": "Cálculo 1"
    },
        {
        "materia": "Projeto Final de Curso 2",
        "dependencia": "Projeto Final de Curso 1"
    }
    ]


    materias_tabela = df[df['codigo_matriz_curricular']=='CICOMP-BI-2']['nome_componente'].drop_duplicates().values

    for idx, req in enumerate(requisitos):
        for key, value in req.items():
            tmp = []
            for materia in materias_tabela:
                lev = levenshtein(value.lower(), materia.lower())
                if lev < 10:
                    tmp.append( [ lev, materia ] )
            tmp = sorted(tmp, key=lambda x: x[0])
            try:
                new_value = tmp[0][1]
                requisitos[idx][key] = new_value
            except:
                pass

    dependencias_grade_antiga = pd.DataFrame(requisitos)
    return dependencias_grade_antiga

def tabela_equivalencias(df):
    materias_novas = df[df['codigo_matriz_curricular']=='CICOMP-BI-3']['nome_componente'].drop_duplicates().values
    materias_antigas = df[df['codigo_matriz_curricular']=='CICOMP-BI-2']['nome_componente'].drop_duplicates().values

    equivalencias = []
    for mn in materias_novas:
        tmp = []
        for ma in materias_antigas:
            lev = levenshtein(mn, ma)
            if lev < 10:
                tmp.append( [levenshtein(mn, ma), ma] )
                tmp = sorted(tmp, key=lambda x: x[0])
        if not tmp:
            continue
        match = tmp[0][1]
        row = {
                "nova": mn,
                "antiga": match
            }
        equivalencias.append(row)
    equivalencias = \
    [{'nova': 'CÁLCULO 1A', 'antiga': 'CÁLCULO 1'},
     {'nova': 'CÁLCULO 2A', 'antiga': 'CÁLCULO 2'},
    #  {'nova': 'FÍSICA III', 'antiga': 'TÓPICOS 1'},
     {'nova': 'ÁLGEBRA LINEAR', 'antiga': 'ÁLGEBRA LINEAR'},
     {'nova': 'PROBABILIDADE E ESTATÍSTICA A',
      'antiga': 'PROBABILIDADE E ESTATÍSTICA'},
    #  {'nova': 'MATEMÁTICA DISCRETA', 'antiga': 'MATEMÁTICA DISCRETA'},
     {'nova': 'PROGRAMAÇÃO ORIENTADA A OBJETOS',
      'antiga': 'PROGRAMAÇÃO ORIENTADA A OBJETOS'},
     {'nova': 'ARQUITETURA DE COMPUTADORES',
      'antiga': 'ARQUITETURA DE COMPUTADORES'},
     {'nova': 'ANÁLISE E PROJETO DE ALGORITMOS',
      'antiga': 'ANÁLISE E PROJETO DE ALGORITMOS'},
     {'nova': 'BANCO DE DADOS', 'antiga': 'BANCO DE DADOS'},
     {'nova': 'COMPUTAÇÃO E SOCIEDADE', 'antiga': 'COMPUTADOR E SOCIEDADE'},
    #  {'nova': 'FÍSICA I', 'antiga': 'TÓPICOS 1'},
     {'nova': 'LÓGICA MATEMÁTICA', 'antiga': 'LÓGICA MATEMÁTICA'},
     {'nova': 'ENGENHARIA DE SOFTWARE', 'antiga': 'ENGENHARIA DE SOFTWARE'},
     {'nova': 'INTRODUÇÃO À PROGRAMAÇÃO', 'antiga': 'INTRODUÇÃO À COMPUTAÇÃO'}]

    manual = [
        {'nova': 'ALGORITMOS E ESTRUTURAS DE DADOS 1', 'antiga': 'ESTRUTURAS DE DADOS I'},
        {'nova': 'ALGORITMOS E ESTRUTURAS DE DADOS 2', 'antiga': 'ESTRUTURAS DE DADOS II'}]
    equivalencias += manual
    equivalencias = pd.DataFrame(equivalencias)
    return equivalencias

In [4]:
tabela_equivalencias(df)

,nova,antiga
0,CÁLCULO 1A,CÁLCULO 1
1,CÁLCULO 2A,CÁLCULO 2
2,ÁLGEBRA LINEAR,ÁLGEBRA LINEAR
3,PROBABILIDADE E ESTATÍSTICA A,PROBABILIDADE E ESTATÍSTICA
4,PROGRAMAÇÃO ORIENTADA A OBJETOS,PROGRAMAÇÃO ORIENTADA A OBJETOS
5,ARQUITETURA DE COMPUTADORES,ARQUITETURA DE COMPUTADORES
6,ANÁLISE E PROJETO DE ALGORITMOS,ANÁLISE E PROJETO DE ALGORITMOS
7,BANCO DE DADOS,BANCO DE DADOS
8,COMPUTAÇÃO E SOCIEDADE,COMPUTADOR E SOCIEDADE
9,LÓGICA MATEMÁTICA,LÓGICA MATEMÁTICA


In [5]:
tabela_dependencias_grade_antiga(df)

,materia,dependencia
0,PROGRAMAÇÃO DE COMPUTADORES 2 CN,PROGRAMAÇÃO DE COMPUTADORES 1 CN
1,ESTRUTURAS DE DADOS I,PROGRAMAÇÃO DE COMPUTADORES 1 CN
2,PROGRAMAÇÃO ORIENTADA A OBJETOS,PROGRAMAÇÃO DE COMPUTADORES 1 CN
3,LINGUAGENS DE PROGRAMAÇÃO,PROGRAMAÇÃO DE COMPUTADORES 1 CN
4,ESTRUTURAS DE DADOS II,ESTRUTURAS DE DADOS I
5,ANÁLISE E PROJETO DE ALGORITMOS,ESTRUTURAS DE DADOS I
6,TEORIA DA COMPUTAÇÃO,LINGUAGENS FORMAIS E AUTÔMATOS
7,COMPILADORES,LINGUAGENS FORMAIS E AUTÔMATOS
8,COMPILADORES,ESTRUTURAS DE DADOS II
9,REDES DE COMPUTADORES 2,REDES DE COMPUTADORES 1


In [6]:
tabela_dependencias_grade_nova(df)

,materia,dependencia
0,CÁLCULO 2A,CÁLCULO 1A
1,ALGORITMOS E ESTRUTURAS DE DADOS 1,INTRODUÇÃO À PROGRAMAÇÃO
2,FÍSICA I,CÁLCULO 1A
3,PROBABILIDADE E ESTATÍSTICA A,CÁLCULO 1A
4,ALGORITMOS E ESTRUTURAS DE DADOS 2,ALGORITMOS E ESTRUTURAS DE DADOS 1
5,MATEMÁTICA DISCRETA,FUNDAMENTOS DE MATEMÁTICA PARA COMPUTAÇÃO
6,PROGRAMAÇÃO ORIENTADA A OBJETOS,INTRODUÇÃO À PROGRAMAÇÃO
7,ARQUITETURA DE COMPUTADORES,FUNDAMENTOS DE MATEMÁTICA PARA COMPUTAÇÃO
8,ANÁLISE E PROJETO DE ALGORITMOS,FUNDAMENTOS DE MATEMÁTICA PARA COMPUTAÇÃO
9,ANÁLISE E PROJETO DE ALGORITMOS,ALGORITMOS E ESTRUTURAS DE DADOS 2


In [42]:
class Aluno(object):
    
    def __init__(self, df, grade_nova=False):
        self.df = df
        self.grade_nova = grade_nova
        self.equivalencias = tabela_equivalencias(df)
        self.deps_antiga = tabela_dependencias_grade_antiga(df)
        self.deps_nova = tabela_dependencias_grade_nova(df)
        self.notas = tabela_notas(df)
        
    def fluxo(self, periodo):
        if self.grade_nova:
            idx = '3'
        else:
            idx = '2'
        tmp = self.df[self.df['codigo_matriz_curricular']== 'CICOMP-BI-' + idx]\
        [['nome_componente', 'semestre_previsto_componente']]
        tmp = tmp[tmp['semestre_previsto_componente'] == periodo]\
        [['nome_componente']]\
        .drop_duplicates().values.flatten().tolist()
        return tmp
    
    def expandir_materias(self, materias):
        tmp = materias[::]
        dct = self.equivalencias.to_dict()
        for key in dct.keys():
            dct[key] = { v:k for k,v in dct[key].items() }
        for materia in materias:
            try:
                key = 'nova'
                other = 'antiga'
                dct[key] = { v:k for k,v in dct[key].items() }
                tmp.append(dct[other][dct[key][materia]])
            except:
                pass
            try:
                key = 'antiga'
                other = 'nova'
                dct[key] = { v:k for k,v in dct[key].items() }
                tmp.append(dct[other][dct[key][materia]])            
            except:
                pass            
        return sorted(list(set(tmp)))
    
    def checar_dependencias(self, materia, cursadas):
        try:
            req_antiga = self.deps_antiga[self.deps_antiga['materia']==materia]\
            .to_dict()['dependencia']\
            .values()
        except:
            req_antiga = []
        try:
            req_nova = self.deps_nova[self.deps_nova['materia']==materia]\
            .to_dict()['dependencia']\
            .values()
        except:
            req_nova = []
        check_antiga = len(set(req_antiga).intersection(set(cursadas))) == len(set(req_antiga))
        check_nova = len(set(req_nova).intersection(set(cursadas))) == len(set(req_nova))
        return check_antiga, check_nova
    
    def criar_fluxo(self, periodos=8, lim=6):
        materias = []
        pendente = []
        materias.extend([ (m,1) for m in self.fluxo(1) ])
        for p in range(2,periodos+1):
            f = self.fluxo(p)
            tmp = pendente[::]
            for item in f:
                tmp.append( [item, self.notas[item], 'not_pending'] )
            tmp = sorted(tmp, key=lambda x: x[1], reverse=True)
            
            for idx,item in enumerate(tmp):
                m = [x[0] for x in materias]
                status_1, status_2 = self.checar_dependencias(item, m)
                if self.grade_nova == False and status_1 == False:
                    tmp[idx][2] = 'blocked'
                if self.grade_nova == True and status_2 == False:
                    tmp[idx][2] = 'blocked'
                if self.grade_nova == False and status_1 == True:
                    tmp[idx][2] = 'not_pending'
                if self.grade_nova == True and status_2 == True:
                    tmp[idx][2] = 'not_pending'    
                    
            blocked = [x for x in tmp if x[2] == 'blocked' ]
            unblocked = [x for x in tmp if x[2] != 'blocked']
            not_pending = unblocked[0:lim]
            pending = unblocked[lim:len(unblocked)]
            for idx,item in enumerate(pending):
                pending[idx][2] = 'pending'
            
            pendente = blocked + pending
            materias.extend([ (m[0],p) for m in not_pending ])
        return materias
    
    def gerar_horas(self, m):
        return self.df[self.df['nome_componente']==m][['ch_total_componente']].values.flatten().tolist()[0]
        pass
    
    def gerar_notas(self, m):
        return self.notas[m]
    
    def gerar_entrada(self, materias, grade_nova=False):
        entrada = []
        atuais = self.expandir_materias(materias)
        materias = []
        materias_novas = []
        materias_antigas = []
        if grade_nova:
            materias_novas = self.df[self.df['codigo_matriz_curricular']== 'CICOMP-BI-3']\
            [['nome_componente']].drop_duplicates().values.flatten().tolist()
        materias_antigas = self.df[self.df['codigo_matriz_curricular']== 'CICOMP-BI-2']\
        [['nome_componente']].drop_duplicates().values.flatten().tolist()
        materias = materias_novas + materias_antigas
        for m in materias:
            status_1, status_2 = self.checar_dependencias(m, atuais)
            if grade_nova == True and status_2 == True and m not in atuais:
                entrada.append(m)
            if grade_nova == False and status_1 == True and m not in atuais:
                entrada.append(m)
            
        notas = [ self.gerar_notas(x) for x in entrada ]
        return pd.DataFrame(list(zip(entrada, notas))).rename(columns={
            0: 'materia',
            1: 'media_geral'
        })
                
a = Aluno(df)
materias = [ x[0] for x in a.criar_fluxo(periodos=3, lim=4) ]

entrada = a.gerar_entrada(materias)
media_global = entrada.mean()[0]

5.751140540097652

In [94]:
from operator import mul
from functools import reduce

class Cromossomo(object):
    
    def __init__(self, entrada):
        self.genes = np.random.randint(2, size=len(entrada.values.tolist()))
        self.X = 2
        self.n_materias = 5
        self.entrada = entrada
    
    # Mostra as matérias que estão sendo sugeridas pelo cromossomo.
    def ler(self):
        rows = self.entrada.values.tolist()
        return [ rows[i][0] for i,v in enumerate(self.genes) if v == 1 ]
        
    #Retorna a fitness do cromossomo.
    def fitness(self):
        N = [ x[1] for x in self.entrada.values.tolist() ]
        selected = [ N[i] for i,v in enumerate(self.genes) if v == 1 ]
        primeiro_termo = 1 / ( np.absolute( np.mean(N) - np.mean(selected)) + 1 )
        
        selected = [ self.X**(v) for i,v in enumerate(self.genes) if v == 1 ]
        segundo_termo = 1 / ( np.absolute(self.X**(self.n_materias) - reduce(mul,selected,1)) + 1 )
        return primeiro_termo * segundo_termo

In [95]:
c = Cromossomo(entrada)
c.genes

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0])

In [96]:
c.ler()

['BANCO DE DADOS',
 'REDES DE COMPUTADORES 1',
 'SISTEMAS OPERACIONAIS 1',
 'ENGENHARIA DE REQUISITOS',
 'COMPUTAÇÃO GRÁFICA',
 'MULTIMÍDIA',
 'SEGURANÇA E AUDITORIA DE SISTEMAS']

In [97]:
c.fitness()

0.007963447226508645

In [9]:
def checar_dependencias(materia, dependencias):
    pass

def aluno_no_fluxo(df, periodo, grade_nova=True, porcentagem=1):
    if grade_nova == True:
        idx = '3'
    else:
        idx = '2'
    tmp = df[df['codigo_matriz_curricular']== 'CICOMP-BI-' + idx]\
    .sort_values(by='semestre_previsto_componente')\
    [['nome_componente', 'semestre_previsto_componente']]
    tmp = tmp[tmp['semestre_previsto_componente'] <= periodo]\
    [['nome_componente']]\
    .drop_duplicates().values.flatten().tolist()
    return tmp
    
# def materias_permitidas(materias_cursadas,
#                         equivalencias,
#                         dependencias_grade_antiga, 
#                         dependencias_grade_nova,
#                         grade_nova=True, 
#                         grade_antiga=True,
#                         optimize=True):
#     eq_dict = {}
#     for item in equivalencias.values.tolist():
#         eq_dict[item[0]] = item[1]
#         eq_dict[item[1]] = item[0]
#     tmp = []
#     for materia in materias_cursadas:
#         try:
#             tmp.append(eq_dict[materia])
#         except:
#             continue
#     materias_cursadas.extend(tmp)
#     materias_cursadas = sorted(list(set(materias_cursadas)))
#     todas_materias = df['nome_componente'].drop_duplicates().values.flatten().tolist()
#     materias_novas = df[df['codigo_matriz_curricular']=='CICOMP-BI-3']['nome_componente'].drop_duplicates().values
#     materias_antigas = df[df['codigo_matriz_curricular']=='CICOMP-BI-2']['nome_componente'].drop_duplicates().values
    
#     permitidas = []
#     if grade_antiga == True:
#         for materia in materias_antigas:

#             dep = dependencias_grade_antiga[dependencias_grade_antiga['materia']==materia]\
#             ['dependencia'].drop_duplicates().values.flatten().tolist()
#             truth_checks = [
#                 len(set(dep).intersection(set(materias_cursadas))) == len(set(dep)),
#                 materia not in materias_cursadas
#             ]
#             if all(truth_checks):
#                 permitidas.append(materia)
            
#     if grade_nova == True:
#         for materia in materias_novas:

#             dep = dependencias_grade_nova[dependencias_grade_nova['materia']==materia]\
#             ['dependencia'].drop_duplicates().values.flatten().tolist()
#             truth_checks = [
#                 len(set(dep).intersection(set(materias_cursadas))) == len(set(dep)),
#                 materia not in materias_cursadas
#             ]
#             if all(truth_checks):
#                 permitidas.append(materia)
        
#     if optimize == True:
#         universo_otimizado = []
#         for item in permitidas:
#             if item not in eq_dict.values():
#                 universo_otimizado.append(item)
#                 continue
#             try:
#                 if eq_dict[item] in permitidas:
#                     n1 = notas[item]
#                     n2 = notas[eq_dict[item]]
#                     if n1 >= n2:
#                         universo_otimizado.append(item)
#                     else:
#                         universo_otimizado.append(eq_dict[item])
#             except:
#                 continue
#         return sorted(list(set(universo_otimizado)))
#     else:
#         return sorted(list(set(permitidas)))

# universo = materias_permitidas(aluno_no_fluxo(df, 2, grade_nova=True),
#                     equivalencias,
#                     dependencias_grade_antiga, 
#                     dependencias_grade_nova)

# entradas = materias_permitidas(aluno_no_fluxo(df, 2, grade_nova=False),
#                     equivalencias, 
#                     dependencias_grade_antiga, 
#                     dependencias_grade_nova,
#                     grade_nova=False,
#                    grade_antiga=True)

# entradas = pd.DataFrame(entradas).rename(columns={
#     0: "nome_componente"
# })

# carga = df[['nome_componente', 'ch_total_componente']].drop_duplicates()
# notas = df[['nome_componente', 'nota_discente_componente']]\
# .groupby('nome_componente')\
# .mean()\
# .reset_index()
# atributos = pd.merge(carga, notas, how='inner', on='nome_componente')
# entradas = pd.merge(atributos, entradas, how='inner', on='nome_componente')
# entradas

In [10]:

# C = 320
# names = wt = entradas['nome_componente'].values.tolist()
# wt = entradas['ch_total_componente'].values.tolist()
# val = entradas['nota_discente_componente'].values.tolist()
# get_items(knapsack(C, wt, val), len(val), C, names)